In [1]:
# # uncomment and run the following command to install the matplotlib 
!pip3 install matplotlib --user

  Using cached https://files.pythonhosted.org/packages/09/03/b7b30fa81cb687d1178e085d0f01111ceaea3bf81f9330c937fb6f6c8ca0/matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7d/2a/2fc11b54e2742db06297f7fa7f420a0e3069fdcf0e4b57dfec33f0b08622/Pillow-8.4.0.tar.gz
  Running setup.py install for pillow ... error
    Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-s89e0ye1/pillow/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-sx11pb7f-record/install-record.txt --single-version-externally-managed --compile --user --prefix=:
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.6
    creating build/lib.linux-x86_64-3.6/PIL
    copying src/PIL/XpmImagePlugin.py -> build/lib.linux-x86_64-3.6/PIL
    copying sr

In [1]:
# importing the required libraries
from pyspark.sql import SparkSession
import pyspark.sql.types as tp
from pyspark.sql import functions as F

# libraries to make plots
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'pyspark'

In [3]:
# create spark session
spark = SparkSession.builder.getOrCreate()

In [4]:
# reading the train data
train_data = spark.read.csv("data/train.csv",inferSchema=True, header=True)

# reading the test data
test_data  = spark.read.csv("data/test.csv", inferSchema=True, header=True)

In [5]:
# data type of the columns
train_data.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [6]:
train_data.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F| 0-17|        10|            A|                         2|             0|                12|              null|              null|    1422

In [7]:
test_data.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|1000004| P00128942|     M|46-50|         7|            B|                         2|             1|                 1|                11|              null|
|1000009| P00113442|     M|26-35|        17|            C|                         0|             0|                 3|                 5|              null|
|1000010| P00288442|     F|36-45|         1|            B|                        4+|             1|                 5|                14|              null|
|1000010| P00145342|     F|36-45|         1|        

In [8]:
# Import functions
from pyspark.sql.functions import countDistinct, avg
from pyspark.sql.types import IntegerType

In [9]:
avgpurchase_amount = train_data.agg(avg("Purchase").cast(IntegerType()))

# 1 Average Purchase Amounts

In [10]:
avgpurchase_amount 

CAST(avg(Purchase) AS INT)
9326


In [11]:
### null values in each column
for c in train_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = train_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 126127
Product_Category_3 282128
Purchase 0


In [12]:
train_data.count()

406568

Considering a lot of values in Product_Category_2 and Product_Category_3 is missing we will remove these values from the testing.

In [13]:
### null values in each column
for c in test_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = test_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 72344
Product_Category_3 162562


# 2. Counting and removing Null Values (delete Product_category_2 and Product_category_3 as they have lot of missing values)

In [14]:
train_data.drop("Product_Category_2","Product_Category_3")

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
1000001,P00069042,F,0-17,10,A,2,0,3,8370
1000001,P00248942,F,0-17,10,A,2,0,1,15200
1000001,P00087842,F,0-17,10,A,2,0,12,1422
1000001,P00085442,F,0-17,10,A,2,0,12,1057
1000002,P00285442,M,55+,16,C,4+,0,8,7969
1000003,P00193542,M,26-35,15,A,3,0,1,15227
1000004,P00184942,M,46-50,7,B,2,1,1,19215
1000004,P00346142,M,46-50,7,B,2,1,1,15854
1000004,P0097242,M,46-50,7,B,2,1,1,15686
1000005,P00274942,M,26-35,20,A,1,1,8,7871


In [15]:
test_data.drop("Product_Category_2","Product_Category_3")

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
1000004,P00128942,M,46-50,7,B,2,1,1
1000009,P00113442,M,26-35,17,C,0,0,3
1000010,P00288442,F,36-45,1,B,4+,1,5
1000010,P00145342,F,36-45,1,B,4+,1,4
1000011,P00053842,F,26-35,1,C,1,0,4
1000013,P00350442,M,46-50,1,C,3,1,2
1000013,P00155442,M,46-50,1,C,3,1,1
1000013,P0094542,M,46-50,1,C,3,1,2
1000015,P00161842,M,26-35,7,A,1,0,10
1000022,P00067942,M,18-25,15,A,4+,0,5


In [16]:
# distinct values in each column
train_data.agg(*(F.countDistinct(F.col(c)).alias(c) for c in train_data.columns))

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
5891,3579,2,7,21,3,5,2,18,17,15,17427


In [17]:

cat_value_Gender = train_data.groupBy("Gender").agg(F.count("Gender"),
                                                            (F.count("Gender")/train_data.count())*100,F.avg("Purchase"))

In [18]:
cat_value_Gender

Gender,count(Gender),((count(Gender) / 406568) * 100),avg(Purchase)
F,100129,24.627860530095827,8806.62969768998
M,306439,75.37213946990417,9495.899781033093


In [19]:
cat_value_Gender_test = test_data.groupBy("Gender").agg(F.count("Gender"),
                                                            (F.count("Gender")/test_data.count())*100)

In [20]:
cat_value_Gender_test

Gender,count(Gender),((count(Gender) / 233599) * 100)
F,57827,24.754814875063676
M,175772,75.24518512493631


# 4 & 5.a Gender Data has more males in the dataset than females. Both test and Train data has roughly 75% males and 25% females . males tend to purchase more in value compared to female per training set but not by a big difference

In [21]:
def cat_value (train_data,x):

    catvalue = train_data.groupBy(x).agg(F.count(x),
                                                         (F.count(x)/train_data.count())*100)
    
  return catvalue

In [22]:
# convert tot udf function
cat_value_udf = F.udf(f = cat_value, returnType= tp.IntegerType())

In [23]:
cat_value_udf(train_data, Age)

In [24]:
cat_value_AGE = train_data.groupBy("Age").agg(F.count("Age"),
                                                            (F.count("Age")/train_data.count())*100,F.avg("Purchase"))

In [25]:
cat_value_AGE

Age,count(Age),((count(Age) / 406568) * 100),avg(Purchase)
18-25,74213,18.25352708526003,9222.336127093637
26-35,161774,39.79014580586765,9299.44379195668
0-17,11327,2.786003817319612,9043.369647744328
46-50,33650,8.276598256626198,9284.202763744428
51-55,28523,7.015554593573523,9626.475651228833
36-45,81133,19.955579386474096,9401.09231755266
55+,15948,3.9225910548788887,9451.037622272384


In [26]:
cat_value_AGE.orderBy('((count(Age) / 406568) * 100)', ascending=False)

Age,count(Age),((count(Age) / 406568) * 100),avg(Purchase)
26-35,161774,39.79014580586765,9299.44379195668
36-45,81133,19.955579386474096,9401.09231755266
18-25,74213,18.25352708526003,9222.336127093637
46-50,33650,8.276598256626198,9284.202763744428
51-55,28523,7.015554593573523,9626.475651228833
55+,15948,3.9225910548788887,9451.037622272384
0-17,11327,2.786003817319612,9043.369647744328


In [27]:
cat_value_AGE_test = test_data.groupBy("Age").agg(F.count("Age"),
                                                            (F.count("Age")/test_data.count())*100)

In [28]:
cat_value_AGE_test

Age,count(Age),((count(Age) / 233599) * 100)
18-25,42293,18.104957641085793
26-35,93428,39.995034225317745
0-17,6232,2.6678196396388683
46-50,19577,8.380600944353358
51-55,16283,6.9704921682027745
36-45,46711,19.996232860585874
55+,9075,3.884862520815586


In [29]:
cat_value_AGE_test.orderBy('((count(Age) / 233599) * 100)', ascending=False)

Age,count(Age),((count(Age) / 233599) * 100)
26-35,93428,39.995034225317745
36-45,46711,19.996232860585874
18-25,42293,18.104957641085793
46-50,19577,8.380600944353358
51-55,16283,6.9704921682027745
55+,9075,3.884862520815586
0-17,6232,2.6678196396388683


# 4&5 b Majority of datapoints is between Age 18-55 in both train and Test data. Most of spending purchases averages around 9-10K range per training date

In [30]:
cat_value_City_Category = train_data.groupBy("City_Category").agg(F.count("City_Category"),
                                                            (F.count("City_Category")/train_data.count())*100,F.avg("Purchase"))

In [31]:
cat_value_City_Category

City_Category,count(City_Category),((count(City_Category) / 406568) * 100),avg(Purchase)
B,171296,42.13218944924343,9188.515400242855
C,125824,30.94783652427146,9846.049632820448
A,109448,26.919974026485114,8943.861696878883


In [32]:
cat_value_City_Category_test = test_data.groupBy("City_Category").agg(F.count("City_Category"),
                                                            (F.count("City_Category")/test_data.count())*100)

In [33]:
cat_value_City_Category_test

City_Category,count(City_Category),((count(City_Category) / 233599) * 100)
B,98566,42.194529942337084
C,72509,31.03994452031045
A,62524,26.76552553735247


# 4&5.c Both test and Train data generally uniformly distributed datapoints for City_Category with slightly larger value for City_Category_B

In [34]:
cat_value_Stay_In_Current_City_Years = train_data.groupBy("Stay_In_Current_City_Years").agg(F.count("Stay_In_Current_City_Years"),
                                                            (F.count("Stay_In_Current_City_Years")/train_data.count())*100,F.avg("Purchase"))

In [35]:
cat_value_Stay_In_Current_City_Years

Stay_In_Current_City_Years,count(Stay_In_Current_City_Years),((count(Stay_In_Current_City_Years) / 406568) * 100),avg(Purchase)
3,70595,17.36363904685071,9337.174488278206
0,55152,13.565258456150017,9234.879623585726
4+,62906,15.472442494244506,9339.196229294503
1,142972,35.165581157395565,9313.550499398483
2,74943,18.4330788453592,9396.003949668415


In [36]:
cat_value_Stay_In_Current_City_Years.orderBy('((count(Stay_In_Current_City_Years) / 406568) * 100)', ascending=False)

Stay_In_Current_City_Years,count(Stay_In_Current_City_Years),((count(Stay_In_Current_City_Years) / 406568) * 100),avg(Purchase)
1,142972,35.165581157395565,9313.550499398483
2,74943,18.4330788453592,9396.003949668415
3,70595,17.36363904685071,9337.174488278206
4+,62906,15.472442494244506,9339.196229294503
0,55152,13.565258456150017,9234.879623585726


In [37]:
cat_value_Stay_In_Current_City_Years_test = test_data.groupBy("Stay_In_Current_City_Years").agg(F.count("Stay_In_Current_City_Years"),
                                                            (F.count("Stay_In_Current_City_Years")/test_data.count())*100)

In [38]:
cat_value_Stay_In_Current_City_Years_test

Stay_In_Current_City_Years,count(Stay_In_Current_City_Years),((count(Stay_In_Current_City_Years) / 233599) * 100)
3,40143,17.184576988771354
0,31318,13.406735474038845
4+,35945,15.387480254624379
1,82604,35.36145274594497
2,43589,18.659754536620447


In [39]:
cat_value_Stay_In_Current_City_Years_test.orderBy('((count(Stay_In_Current_City_Years) / 233599) * 100)', ascending=False)

Stay_In_Current_City_Years,count(Stay_In_Current_City_Years),((count(Stay_In_Current_City_Years) / 233599) * 100)
1,82604,35.36145274594497
2,43589,18.659754536620447
3,40143,17.184576988771354
4+,35945,15.387480254624379
0,31318,13.406735474038845


# 4 & 5.d People who are staying in year 1-2 is 35% percent for both test and Train Data where as all the other population is 13-18% of the data and all purchases are around the same range in train data

In [40]:
cat_value_Marital_Status = train_data.groupBy("Marital_Status").agg(F.count("Marital_Status"),
                                                            (F.count("Marital_Status")/train_data.count())*100,F.avg("Purchase"))

In [41]:
cat_value_Marital_Status

Marital_Status,count(Marital_Status),((count(Marital_Status) / 406568) * 100),avg(Purchase)
1,166777,41.02069026583499,9331.293679584116
0,239791,58.97930973416501,9322.567952925672


In [42]:
cat_value_Marital_Status_test = test_data.groupBy("Marital_Status").agg(F.count("Marital_Status"),
                                                            (F.count("Marital_Status")/test_data.count())*100)

In [43]:
cat_value_Marital_Status_test

Marital_Status,count(Marital_Status),((count(Marital_Status) / 233599) * 100)
1,95792,41.007024858839294
0,137807,58.992975141160706


# 4&5 E the Data is evenly distributed among married and non-married folks  with non married being slightly higher at 58% compared to 42% of married

In [44]:
# importing pre-processing libraries
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator

In [45]:
# label encode Age
SI_Age = StringIndexer(inputCol= "Age", outputCol= "Age_le" , handleInvalid="skip")

# label encode Gender
SI_Gender = StringIndexer(inputCol= "Gender", outputCol= "Gender_le", handleInvalid= "skip")

# label encode Stay_In_Current_City_Years
SI_Stay_In_Current_City_Years      = StringIndexer(inputCol= "Stay_In_Current_City_Years",      outputCol= "Stay_In_Current_City_Years_le", handleInvalid= "skip")

# label encode City_Category
SI_City_Category = StringIndexer(inputCol= "City_Category", outputCol= "City_Category_le", handleInvalid="skip")

In [46]:
# label encode object Age
SI_Age_obj = SI_Age.fit(train_data)

# label encode object Gender
SI_Gender_Obj = SI_Gender.fit(train_data)

# label encode object stay_in_current_years
SI_Stay_In_Current_City_Years_Obj = SI_Stay_In_Current_City_Years.fit(train_data)

# label encode object city_category
SI_City_Category_Obj = SI_City_Category.fit(train_data)

In [47]:
# label encode country
train_data_encoded = SI_Age_obj.transform(train_data)

# label encode browser
train_data_encoded = SI_Gender_Obj.transform(train_data_encoded)

# label encode os
train_data_encoded = SI_Stay_In_Current_City_Years_Obj.transform(train_data_encoded)

# label encode traffic
train_data_encoded = SI_City_Category_Obj.transform(train_data_encoded)

In [48]:
train_data_encoded.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Age_le',
 'Gender_le',
 'Stay_In_Current_City_Years_le',
 'City_Category_le']

In [49]:
train_data_encoded.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+------+---------+-----------------------------+----------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|Age_le|Gender_le|Stay_In_Current_City_Years_le|City_Category_le|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+------+---------+-----------------------------+----------------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|   6.0|      1.0|                          1.0|             2.0|
|1000001| P00248942|     F| 0-17|        10|            A|                  

In [50]:
# label encode Age
# test data
test_data_encoded = SI_Age_obj.transform(test_data)

# label encode Gender

# test data
test_data_encoded = SI_Gender_Obj.transform(test_data_encoded)

# label encode current_city_Years
# test data
test_data_encoded = SI_Stay_In_Current_City_Years_Obj.transform(test_data_encoded)

# label encode city_category
test_data_encoded = SI_City_Category_Obj.transform(test_data_encoded)

In [51]:
test_data_encoded.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+------+---------+-----------------------------+----------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Age_le|Gender_le|Stay_In_Current_City_Years_le|City_Category_le|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+------+---------+-----------------------------+----------------+
|1000004| P00128942|     M|46-50|         7|            B|                         2|             1|                 1|                11|              null|   3.0|      0.0|                          1.0|             0.0|
|1000009| P00113442|     M|26-35|        17|            C|                         0|             0|            

In [52]:
OHE_train = OneHotEncoderEstimator(inputCols=["Gender_le",
                                              "City_Category_le",
                                              "Occupation"],
                                  outputCols=["Gender_ohe",
                                              "City_Category_ohe",
                                              "Occupation_ohe"])

In [53]:
# OHE object
OHE_Obj = OHE_train.fit(train_data_encoded)

In [54]:
# Transform train data
train_data_encoded = OHE_Obj.transform(train_data_encoded)

In [55]:
# view the one hot encoded data
train_data_encoded.select("Gender_le",
                          "City_Category_le", 
                          "Occupation",
                          "Gender_ohe",
                          "City_Category_ohe",
                          "Occupation_ohe").show()

+---------+----------------+----------+-------------+-----------------+---------------+
|Gender_le|City_Category_le|Occupation|   Gender_ohe|City_Category_ohe| Occupation_ohe|
+---------+----------------+----------+-------------+-----------------+---------------+
|      1.0|             2.0|        10|    (1,[],[])|        (2,[],[])|(20,[10],[1.0])|
|      1.0|             2.0|        10|    (1,[],[])|        (2,[],[])|(20,[10],[1.0])|
|      1.0|             2.0|        10|    (1,[],[])|        (2,[],[])|(20,[10],[1.0])|
|      1.0|             2.0|        10|    (1,[],[])|        (2,[],[])|(20,[10],[1.0])|
|      0.0|             1.0|        16|(1,[0],[1.0])|    (2,[1],[1.0])|(20,[16],[1.0])|
|      0.0|             2.0|        15|(1,[0],[1.0])|        (2,[],[])|(20,[15],[1.0])|
|      0.0|             0.0|         7|(1,[0],[1.0])|    (2,[0],[1.0])| (20,[7],[1.0])|
|      0.0|             0.0|         7|(1,[0],[1.0])|    (2,[0],[1.0])| (20,[7],[1.0])|
|      0.0|             0.0|    

In [56]:
# transform the data
# test data
test_data_encoded = OHE_Obj.transform(test_data_encoded)

In [57]:
test_data_encoded.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+------+---------+-----------------------------+----------------+-------------+-----------------+---------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Age_le|Gender_le|Stay_In_Current_City_Years_le|City_Category_le|   Gender_ohe|City_Category_ohe| Occupation_ohe|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+------+---------+-----------------------------+----------------+-------------+-----------------+---------------+
|1000004| P00128942|     M|46-50|         7|            B|                         2|             1|                 1|                11|              null|   3.0|      0.0|                

# 8 Baseline model wil be a Linear regression model since this is a regression problem trying to find the purchase power of the customer

In [58]:
train_data_encoded.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Age_le',
 'Gender_le',
 'Stay_In_Current_City_Years_le',
 'City_Category_le',
 'Gender_ohe',
 'City_Category_ohe',
 'Occupation_ohe']

So, now we will select the features and create a vector using VectorAssembler

Gender_ohe (One Hot Encoded)
Age_le (Label Encode)
Occupation_ohe (One-hot encode)
City category_ohe (One-hot encode)
stay in current city years_le (Label encode)
Product_category_1 (As is)
Marital_status (As is)

In [59]:
# import the library
from pyspark.ml.feature import VectorAssembler

# create feature vector
feature_vector = VectorAssembler(inputCols= ['Gender_ohe',
                                             'Age_le',
                                             'Occupation_ohe',
                                             'City_Category_ohe',
                                             'Stay_In_Current_City_Years_le',
                                             'Product_Category_1',
                                             'Marital_Status'],
                                outputCol= 'feature_vector')

In [60]:
# transform the feature vector
train_data_encoded = feature_vector.transform(train_data_encoded)

In [61]:
# view the feature vector
train_data_encoded.select("feature_vector").show(10)

+--------------------+
|      feature_vector|
+--------------------+
|(27,[1,12,24,25],...|
|(27,[1,12,24,25],...|
|(27,[1,12,24,25],...|
|(27,[1,12,24,25],...|
|(27,[0,1,18,23,24...|
|(27,[0,17,24,25],...|
|(27,[0,1,9,22,24,...|
|(27,[0,1,9,22,24,...|
|(27,[0,1,9,22,24,...|
|(27,[0,25,26],[1....|
+--------------------+
only showing top 10 rows



In [62]:
# we can see that the data type of the `feature_vector` is vector
train_data_encoded.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)
 |-- Age_le: double (nullable = false)
 |-- Gender_le: double (nullable = false)
 |-- Stay_In_Current_City_Years_le: double (nullable = false)
 |-- City_Category_le: double (nullable = false)
 |-- Gender_ohe: vector (nullable = true)
 |-- City_Category_ohe: vector (nullable = true)
 |-- Occupation_ohe: vector (nullable = true)
 |-- feature_vector: vector (nullable = true)



In [63]:
# test data
test_data_encoded = feature_vector.transform(test_data_encoded)

In [64]:
# importing the libraries
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [65]:
LR = LinearRegression(featuresCol='feature_vector', labelCol="Purchase")

In [66]:
model_LR = LR.fit(train_data_encoded)

In [67]:
from pyspark.ml.evaluation import RegressionEvaluator

In [68]:
LR_evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse")
LR_evaluator.evaluate(model_LR.transform(train_data_encoded)) 

4703.368102509055

In [70]:
lr_predictions = model_LR.transform(test_data_encoded)

In [71]:
lr_predictions.select("prediction","feature_vector").show(5)

+------------------+--------------------+
|        prediction|      feature_vector|
+------------------+--------------------+
|11262.008637833615|(27,[0,1,9,22,24,...|
| 11059.64344411907|(27,[0,19,23,24,2...|
| 8778.288556130501|(27,[1,3,22,24,25...|
| 9189.651922156107|(27,[1,3,22,24,25...|
| 9757.713020964482|(27,[3,23,25],[1....|
+------------------+--------------------+
only showing top 5 rows



In [72]:
# import the CrossValidator and ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [73]:
# create the ParamGridBuilder
params = ParamGridBuilder().build()

In [74]:
# create the object of the Logistic Regression Model
model_LR_CV = LinearRegression(featuresCol= "feature_vector",  labelCol="Purchase")

In [75]:
# create object of the cross validation model with numFolds = 3
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=params,
                    evaluator=LR_evaluator,
                    numFolds=3,
                    seed=27)

In [76]:
## fit the model
cv_model = cv.fit(train_data_encoded)

In [77]:
# evaluate the model
LR_evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse")
LR_evaluator.evaluate(model_LR.transform(train_data_encoded)) 

4703.368102509055

In [78]:
# evaluate model on validation data
# LR_evaluator.evaluate(cv_model.transform(test_data_encoded))

In [79]:
# create parameter builder

updated_params = ParamGridBuilder() \
                .addGrid(model_LR_CV.regParam, [0.01, 0.005, 0.0001]) \
                .addGrid(model_LR_CV.elasticNetParam, [0.1, 0.001]) \
                .build()

In [80]:
# create object of the Cross Calidator with 3 folds
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=updated_params,
                    evaluator=LR_evaluator,
                    numFolds=3,
                    seed=27)

In [81]:
# fit the model
grid_model = cv.fit(train_data_encoded)

In [82]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse")
LR_evaluator.evaluate(model_LR.transform(train_data_encoded)) 

4703.368102509055

In [83]:
# extract the best model parameters dictionary
param_dict = grid_model.bestModel.extractParamMap()

In [84]:
param_dict


{Param(parent='LinearRegression_5ec2e1e65300', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LinearRegression_5ec2e1e65300', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.1,
 Param(parent='LinearRegression_5ec2e1e65300', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'): 1.35,
 Param(parent='LinearRegression_5ec2e1e65300', name='featuresCol', doc='features column name'): 'feature_vector',
 Param(parent='LinearRegression_5ec2e1e65300', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LinearRegression_5ec2e1e65300', name='labelCol', doc='label column name'): 'Purchase',
 Param(parent='LinearRegression_5ec2e1e65300', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'): 'squaredError',


In [85]:
# created a filtered dictionary
final_dict = {}
for k, v in param_dict.items():
    final_dict[k.name] = v

In [86]:
# get the best elastic net parameter
final_dict["elasticNetParam"]

0.1

In [87]:
# get the best regularization parameter
final_dict["regParam"]

0.01

# Pipe line building

In [88]:
train_data_pipeline = spark.read.csv("data/train.csv", inferSchema=True, header=True)
test_data_pipeline = spark.read.csv("data/test.csv", inferSchema=True, header=True)

In [89]:
train_data_pipeline = train_data_pipeline.withColumn("Purchase", F.col("Purchase").cast(tp.IntegerType()))

# Pipelines

Stage 1 Remove Columns Product_Category_2, Product_Category_3 from the dataset.

Stage 2 Label Encode: Age

Stage 3 Label Encode: Gender

Stage 4 Label Encode: Stay_In_Current_City_Years

Stage 5 Label Encode: City_Category

Stage 6 OHE: Gender, CIty_Category, Occupation

Stage 7 Create Vector (Gender_ohe', 'Age_le', 'Occupation_ohe', 'City_Category_ohe', 'Stay_In_Current_City_Years_le', 'Product_Category_1', 'Marital_Status')

STAGE 8: [Estimator] Predict Label Using the Linear Regression

In [90]:
## Import the Required Libraries 
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.ml import classification
from pyspark.ml.regression import LinearRegression

In [91]:
# custom transformer to drop Product_category_2 & Product_category_3

class nullValuesTransformer(Transformer):
    
    def __init__(self, dataframe = None):
        self.dataframe = dataframe
    
    def _transform(self, dataframe):
        dataframe = dataframe.drop("Product_Category_2","Product_Category_3")

        
        return dataframe

In [92]:
# Stage 1 - drop null values
stage_1 = nullValuesTransformer()

# Stage 2 - label encode Age
stage_2 = StringIndexer(inputCol= "Age", outputCol= "Age_le") 

# Stage 3 - label encode Gender
stage_3 = StringIndexer(inputCol= "Gender", outputCol= "Gender_le")

# Stage 4 - label encode Stay_In_Current_City_Years
stage_4 = StringIndexer(inputCol= "Stay_In_Current_City_Years", outputCol= "Stay_In_Current_City_Years_le")

# Stage 5 - label encode City_Category
stage_5 = StringIndexer(inputCol= "City_Category", outputCol= "City_Category_le")

 # Stage 6 - One Hot Encode columns
stage_6 = OneHotEncoderEstimator(inputCols= ["Gender_le",  "City_Category_le", "Occupation"], 
                        outputCols= ["Gender_ohe",  "City_Category_ohe", "Occupation_ohe"])

# Stage 7 - Create vector from the columns
stage_7 = VectorAssembler(inputCols= ['Gender_ohe',
                                             'Age_le',
                                             'Occupation_ohe',
                                             'City_Category_ohe',
                                             'Stay_In_Current_City_Years_le',
                                             'Product_Category_1',
                                             'Marital_Status'],
                         outputCol=  "feature_vector")

# Stage 8 - Train ML model
stage_8 = LinearRegression(featuresCol='feature_vector', labelCol="Purchase")

In [93]:
# Define pipeline
pipeline = Pipeline(stages= [stage_1,
                             stage_2,
                             stage_3,
                             stage_4,
                             stage_5,
                             stage_6,
                             stage_7,
                             stage_8,
                             ])

In [94]:
# fit the pipeline with the training data
pipeline_model = pipeline.fit(train_data_pipeline)

# transform data
final_data = pipeline_model.transform(train_data_pipeline)

In [95]:
final_data.select("Gender_ohe", "Age_le", "Purchase").show(10)

+-------------+------+--------+
|   Gender_ohe|Age_le|Purchase|
+-------------+------+--------+
|    (1,[],[])|   6.0|    8370|
|    (1,[],[])|   6.0|   15200|
|    (1,[],[])|   6.0|    1422|
|    (1,[],[])|   6.0|    1057|
|(1,[0],[1.0])|   5.0|    7969|
|(1,[0],[1.0])|   0.0|   15227|
|(1,[0],[1.0])|   3.0|   19215|
|(1,[0],[1.0])|   3.0|   15854|
|(1,[0],[1.0])|   3.0|   15686|
|(1,[0],[1.0])|   0.0|    7871|
+-------------+------+--------+
only showing top 10 rows



In [96]:
final_test_data = pipeline_model.transform(test_data_pipeline)

In [97]:
final_test_data.select("Gender_ohe", "Age_le", "prediction").show(10)

+-------------+------+------------------+
|   Gender_ohe|Age_le|        prediction|
+-------------+------+------------------+
|(1,[0],[1.0])|   3.0|11262.008637833615|
|(1,[0],[1.0])|   0.0| 11059.64344411907|
|    (1,[],[])|   1.0| 8778.288556130501|
|    (1,[],[])|   1.0| 9189.651922156107|
|    (1,[],[])|   0.0| 9757.713020964482|
|(1,[0],[1.0])|   3.0|11154.673800946994|
|(1,[0],[1.0])|   3.0|11566.037166972601|
|(1,[0],[1.0])|   3.0|11154.673800946994|
|(1,[0],[1.0])|   0.0| 7292.822625533465|
|(1,[0],[1.0])|   2.0| 9702.928815697838|
+-------------+------+------------------+
only showing top 10 rows

